# Domain Sorting 
## Author: Damien 

For this implementation, of domain sorting, we assume that the domain and all sub-domains are hyperrectangles.

In [22]:
import pickle
import numpy as np
from anytree import NodeMixin, RenderTree

In [44]:
class DomainTree:
    """
    Author:         Damien Beecroft
    Domain_Tree:    A super of the Domain class that stores important functions for working with
                    the domain decomposition tree.
    """
    def __init__(self):
        self.dnn_size = [3,80,2]
        self.nonlin_size = [3,80,2]
        self.lin_size = [3,4,2]
    
    def find_interior_pts(self): # find which of the parent's points are in the current domain
        parent_pts = self.parent.pts # get the parent's points
        verts = self.vertices
        mask = [((verts[0] <= pt) & (pt <= verts[1])).all() for pt in parent_pts] # find which of the parent's points are in the current domain
        domain_pts = parent_pts[mask]
        return domain_pts

class Domain(DomainTree,NodeMixin):
    """
    Author:     Damien Beecroft
    Domain:     A class that defines properties of the relevant domains used for tracking where each
                neural network has support. The local properties of the nodes are defined here. The "global"
                variables are stored in the support class: DomainTree.
    =================================================================================================
    vertices:   Two opposite points on the hyperrectangle used to define the domain. 
    parent:     The node corresponding to the domain that the current node's domain is the immediate subset of
    children:   The nodes whose domains are the immediate subsets of the current domain.
    pts:        List of collocation points for the neural network to be evaluated at. This value is only passed 
                in for the root node. All other nodes determine the points determine their interior points through
                the find_interior points function in the Domain_Tree class.
    net_sizes:  [base net shape, nonlinear net shape, linear net shape]. The base net is the 
    """
    def __init__(self,vertices,parent=None,children=None,pts=None):
        super(Domain,self).__init__()
        self.vertices = vertices # two opposite vertices that define the n-dimensional box
        self.parent = parent # parent domain of the current domain
        self.dnn = None
        self.nonlin_net = None
        self.lin_net = None
        if parent: # set level
            self.lvl = parent.lvl + 1
            self.pts = self.find_interior_pts() # otherwise, find which points are in the domain of the current domain
            self.lin_net = np.zeros(self.lin_size)
            self.nonlin_net = np.zeros(self.lin_size)            
        else:
            self.lvl = 0
            self.pts = pts # if this is the root node, just assign points
            self.dnn = np.zeros(self.dnn_size)

        if children: # set children
            self.children = children

### Can you make a tree where the nodes are two distinct types of classes?

In [55]:
class Fomain(DomainTree,NodeMixin):
    def __init__(self,dummy_var,parent=None,children=None):
        super(Fomain,self).__init__()
        self.dummy = dummy_var
        self.parent = parent # parent domain of the current domain

        if parent: # set level
            self.lvl = parent.lvl + 1
        else:
            self.lvl = 0

        if children: # set children
            self.children = children

In [56]:
dom0 = np.array([[0.],[1.]])
dom10 = np.array([[0.],[0.6]])
dom11 = np.array([[0.4],[1.]])
dom20 = np.array([[0.],[0.35]])
dom21 = np.array([[0.25],[0.6]])

D0 = Domain(dom0,pts=np.array([[0.2],[0.6],[0.9],[0.4],[0.8]]))
D10 = Domain(dom10,parent=D0)
D11 = Domain(dom10,parent=D0)
F12 = Fomain(3,parent=D0) # Adding new dummy class to tree
D20 = Domain(dom20,parent=D10)
D21 = Domain(dom20,parent=D10)


In [58]:
F12.parent
# Eureka!